In [1]:
import numpy as np
import tensorflow as tf
from PIL import Image
import os
import random
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim, peak_signal_noise_ratio as psnr
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Define the model path
model_path = 'cycle_gan_generator_g2.h5'

# Load the saved generator model
generator = tf.keras.models.load_model(model_path)

# Example of compiling the model manually
generator.compile(optimizer='adam')  # Or use any optimizer you like

# Function to load and preprocess images
def load_and_preprocess_image(img_path):
    img = Image.open(img_path).convert("RGB")
    img = img.resize((128, 128))  # Resize to match the input size of the model
    img_array = np.array(img) / 255.0  # Normalize to [0, 1]
    return img_array

# Define paths
test_dir = r"C:\Users\aruns\Downloads\test"
real_image_dir = os.path.join(test_dir, "real")
cartoon_dir = os.path.join(test_dir, "trainB")

# Get a list of image files
real_image_files = [os.path.join(real_image_dir, fname) for fname in os.listdir(real_image_dir)]
cartoon_images = [os.path.join(cartoon_dir, fname) for fname in os.listdir(cartoon_dir)]

# Randomly select 10 images
selected_indices = random.sample(range(len(real_image_files)), 5)
selected_real_images = [real_image_files[i] for i in selected_indices]
selected_cartoon_images = [cartoon_images[i] for i in selected_indices]

# Visualize the images
fig, axs = plt.subplots(5, 3, figsize=(15, 50))

mse_scores = []
mae_scores = []
ssim_scores = []
psnr_scores = []

for i, (real_img_path, cartoon_img_path) in enumerate(zip(selected_real_images, selected_cartoon_images)):
    # Load and preprocess images
    real_img = load_and_preprocess_image(real_img_path)
    expected_cartoon_img = load_and_preprocess_image(cartoon_img_path)
    
    # Generate cartoon image
    real_img_batch = np.expand_dims(real_img, axis=0)
    generated_cartoon_img = generator.predict(real_img_batch)[0]
    
    # Plot real image
    axs[i, 0].imshow(real_img)
    axs[i, 0].set_title("Real Image")
    axs[i, 0].axis("off")
    
    # Plot expected cartoon image
    axs[i, 1].imshow(expected_cartoon_img)
    axs[i, 1].set_title("Expected Cartoon")
    axs[i, 1].axis("off")
    
    # Plot generated cartoon image
    axs[i, 2].imshow(generated_cartoon_img)
    axs[i, 2].set_title("CycleGAN Generated Cartoon")
    axs[i, 2].axis("off")
    
    # Calculate similarity metrics
    mse = mean_squared_error(expected_cartoon_img.flatten(), generated_cartoon_img.flatten())
    mse_scores.append(mse)

    mae = mean_absolute_error(expected_cartoon_img.flatten(), generated_cartoon_img.flatten())
    mae_scores.append(mae)
    
    ssim_score = ssim(expected_cartoon_img, generated_cartoon_img, win_size=11, channel_axis=-1, data_range=1.0)
    ssim_scores.append(ssim_score)
    
    psnr_score = psnr(expected_cartoon_img, generated_cartoon_img, data_range=1.0)
    psnr_scores.append(psnr_score)
    
    print(f"Image {i+1} - MSE: {mse:.4f}, MAE: {mae:.4f}, SSIM: {ssim_score:.4f}, PSNR: {psnr_score:.4f}")

plt.tight_layout()
plt.show()


Image 1 - MSE: 0.1239, MAE: 0.2813, SSIM: 0.0672, PSNR: 9.0688
Image 2 - MSE: 0.1409, MAE: 0.3097, SSIM: 0.0539, PSNR: 8.5111
Image 3 - MSE: 0.0759, MAE: 0.2231, SSIM: 0.0494, PSNR: 11.1967
Image 4 - MSE: 0.0918, MAE: 0.2529, SSIM: 0.0409, PSNR: 10.3721
Image 5 - MSE: 0.0676, MAE: 0.2143, SSIM: 0.0503, PSNR: 11.7003


In [2]:
import numpy as np
import tensorflow as tf
from PIL import Image
import os
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim, peak_signal_noise_ratio as psnr
from sklearn.metrics import mean_squared_error

# Define the model path
model_path = 'cycle_gan_generator_g2.h5'

# Load the saved generator model
generator = tf.keras.models.load_model(model_path)

# Example of compiling the model manually
generator.compile(optimizer='adam')  # Or use any optimizer you like

# Function to load and preprocess images
def load_and_preprocess_image(img_path):
    img = Image.open(img_path).convert("RGB")
    img = img.resize((128, 128))  # Resize to match the input size of the model
    img_array = np.array(img) / 255.0  # Normalize to [0, 1]
    return img_array

# Define paths
test_dir = r"C:\Users\aruns\Downloads\test"
real_image_dir = os.path.join(test_dir, "real")
cartoon_dir = os.path.join(test_dir, "trainB")

# Get a list of image files
real_images = [os.path.join(real_image_dir, fname) for fname in os.listdir(real_image_dir) if fname.lower().endswith(('png', 'jpg', 'jpeg'))]
cartoon_images = [os.path.join(cartoon_dir, fname) for fname in os.listdir(cartoon_dir) if fname.lower().endswith(('png', 'jpg', 'jpeg'))]

# Check that both directories have the same number of images
assert len(real_images) == len(cartoon_images), "The number of images in both directories must match."

# Lists to store metric values
mse_scores = []
mae_scores = []
ssim_scores = []
psnr_scores = []

for real_img_path, cartoon_img_path in zip(real_images, cartoon_images):
    # Load and preprocess images
    real_img = load_and_preprocess_image(real_img_path)
    expected_cartoon_img = load_and_preprocess_image(cartoon_img_path)
    
    # Generate cartoon image
    real_img_batch = np.expand_dims(real_img, axis=0)
    generated_cartoon_img = generator.predict(real_img_batch)[0]
    
    # Ensure the generated image is in the correct format
    generated_cartoon_img = np.clip(generated_cartoon_img, 0, 1)
    
    # Convert images to proper format for metrics
    real_img_uint8 = (real_img * 255).astype(np.uint8)
    generated_cartoon_img_uint8 = (generated_cartoon_img * 255).astype(np.uint8)
    expected_cartoon_img_uint8 = (expected_cartoon_img * 255).astype(np.uint8)
    
    # Calculate similarity metrics
    mse = mean_squared_error(expected_cartoon_img.flatten(), generated_cartoon_img.flatten())
    mse_scores.append(mse)

    mae = mean_absolute_error(expected_cartoon_img.flatten(), generated_cartoon_img.flatten())
    mae_scores.append(mae)
    
    ssim_score = ssim(expected_cartoon_img, generated_cartoon_img, channel_axis=-1, data_range=1.0)
    ssim_scores.append(ssim_score)
    
    psnr_score = psnr(expected_cartoon_img_uint8, generated_cartoon_img_uint8)
    psnr_scores.append(psnr_score)
    

# Compute averages
average_mse = np.mean(mse_scores)
average_mae = np.mean(mae_scores)
average_ssim = np.mean(ssim_scores)
average_psnr = np.mean(psnr_scores)

# Print average metrics
print(f"\nAverage MSE: {average_mse:.4f}")
print(f"Average MAE: {average_mae:.4f}")
print(f"Average SSIM: {average_ssim:.4f}")
print(f"Average PSNR: {average_psnr:.4f}")



Average MSE: 0.1278
Average MAE: 0.2902
Average SSIM: 0.0653
Average PSNR: 9.1249
